# SummaryIndex

摘要索引：

- 简单的数据结构，节点存储在列表中
- 索引过程中，文档被拆分为节点，存储在列表中
- 查询过程，可以通过参数过滤，并将所有节点合成为回答

In [1]:
%%time

items=[
    "颐和园",
]

from llama_index.readers.web import TrafilaturaWebReader

documents = TrafilaturaWebReader().load_data(
    [ f"https://baike.baidu.com/item/{item}" for item in items]
)

len(documents)

CPU times: user 3.13 s, sys: 394 ms, total: 3.52 s
Wall time: 3.54 s


1

## 创建索引和检索器

In [8]:
%%time

from llama_index.core import Settings

Settings.chunk_size = 128
Settings.chunk_overlap = 10

nodes = Settings.node_parser.get_nodes_from_documents(documents)
nodes[0]

CPU times: user 44.3 ms, sys: 1 µs, total: 44.3 ms
Wall time: 43.2 ms


TextNode(id_='66df8f68-c2f8-4bd8-86da-29f6524ff58f', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='https://baike.baidu.com/item/颐和园', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='168322b9cad4204cbf8cbc77565c9649e660fa554d2f1631473b641a9fa66d5b'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='9fa1b885-9bed-48e9-9d38-899bb9fa9452', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='994ac7a04fdf52b0204cfb450624d5a98ee22b5b768555df61ddc271374c32c7')}, text='收藏\n查看我的收藏\n0有用+1\n颐和园，中国清朝时期皇家园林，前身为清漪园，位于北京市西郊海淀区新建宫门路19号。 [52] [72]其始建于清乾隆十五年（1750年），咸丰十年（1860年）被英法联军烧毁。', start_char_idx=0, end_char_idx=104, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')

In [11]:
len(nodes)

266

In [9]:
%%time

from llama_index.core import SummaryIndex
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.embed_model = OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://ape:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

summary_index = SummaryIndex(nodes)
retriever=summary_index.as_retriever()
results=retriever.retrieve("总结下颐和园的历史")

len(results)

CPU times: user 39.9 ms, sys: 0 ns, total: 39.9 ms
Wall time: 39.2 ms


266

In [10]:
results[0]

NodeWithScore(node=TextNode(id_='66df8f68-c2f8-4bd8-86da-29f6524ff58f', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='https://baike.baidu.com/item/颐和园', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='168322b9cad4204cbf8cbc77565c9649e660fa554d2f1631473b641a9fa66d5b'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='9fa1b885-9bed-48e9-9d38-899bb9fa9452', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='994ac7a04fdf52b0204cfb450624d5a98ee22b5b768555df61ddc271374c32c7')}, text='收藏\n查看我的收藏\n0有用+1\n颐和园，中国清朝时期皇家园林，前身为清漪园，位于北京市西郊海淀区新建宫门路19号。 [52] [72]其始建于清乾隆十五年（1750年），咸丰十年（1860年）被英法联军烧毁。', start_char_idx=0, end_char_idx=104, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)

## 使用检索查询

In [13]:
%%time

from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.llms.openai_like import OpenAILike

Settings.llm=OpenAILike(
    model="qwen2", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# query
response = query_engine.query("总结下颐和园的历史")
print(response)

颐和园是中国清朝时期的皇家园林，位于北京西郊。其创建始于乾隆十五年（1750年），作为清朝皇帝的避暑行宫，融合了江南园林风格，并在乾隆年间进行了大规模扩建。咸丰十年（1860年）期间，在第二次鸦片战争中，颐和园遭到英法联军的焚掠，大量建筑被破坏，文物损失惨重。战后，清朝国力衰弱，颐和园的陈设有所减少。光绪十二年（1886年），慈禧太后下令重建并扩大清漪园，并于两年后改名为颐和园。在重建过程中，大量珍宝被搜集用于园内文物陈设，达到极盛状态。八国联军攻入北京后（1900年），颐和园再次遭受破坏。

新中国成立后，尤其是改革开放以来，颐和园的文物保护工作得到了重视，并进行了多次修复。颐和园于1961年被列入中国第一批全国重点文物保护单位，并在1998年作为“清西苑、北海公园、中南海、颐和园”被列为世界文化遗产。

综上所述，颐和园经历了从创建、扩建到两次战火的破坏，再到重建和现代化保护的过程。它不仅是中国园林艺术的瑰宝，也是研究清朝历史、建筑、文化和环保的重要窗口。
CPU times: user 297 ms, sys: 4.49 ms, total: 302 ms
Wall time: 1min 34s


## 比较向量索引的总结效果

In [14]:
%%time

from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(nodes)
retriever=vector_index.as_retriever()

results=retriever.retrieve("总结下颐和园的历史")
len(results)

CPU times: user 1.18 s, sys: 62.3 ms, total: 1.24 s
Wall time: 26.4 s


2

In [15]:
results[0]

NodeWithScore(node=TextNode(id_='b5195d78-6224-49af-bb07-03b74b88d0fb', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='https://baike.baidu.com/item/颐和园', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='168322b9cad4204cbf8cbc77565c9649e660fa554d2f1631473b641a9fa66d5b'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='d6002527-27d2-4dcb-8a81-4583d0207a32', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='ff2b4ad94d60006feae82aff10cb52aa65c751555d67ad0f7592b8b639313185'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='5d98906b-c78f-416d-9efb-c0e3ba70bf94', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='56f84f742087af6bfd8d4cce154fe368db921d94486c2701e591e2c3ddeab02b')}, text='并用了5年时间，将殿堂内装修、匾联和一切陈设进行修整，增加了解说，进行爱国主义的宣传和教育。利用颐和园古建筑，供大众休养和举办各种活动。\n1954年，完成以全园中心建筑佛香阁为主的整修油饰。', start_char_idx=5558, end_char_idx=5653, text_template='{metadat

In [16]:
%%time

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# query
response = query_engine.query("总结下颐和园的历史")
print(response)

颐和园的历史可以总结为以下几个关键点：

1. **5年装修与修整**：颐和园在某个时期，耗时5年对殿堂内的装修、匾联以及所有陈设进行了全面的修整，并增加了解说内容。这一举措旨在进行爱国主义的宣传和教育。

2. **佛香阁为主的整修油饰完成**：1954年，完成了以全园中心建筑佛香阁为主的大规模整修与油饰工作。

3. **成立中国共产党颐和园委员会**：1972年，颐和园成立了中国共产党颐和园委员会。同年5月1日，根据周恩来总理的指示，开始恢复殿堂内的文物陈列，并逐步恢复了古建筑上的苏式彩画、匾联以及养花活动。

4. **全面加强对文物管理**：随着上述活动的开展，也结合清仓核资工作，对颐和园的文物进行了全面加强的管理和保护。
CPU times: user 20.3 ms, sys: 6 µs, total: 20.4 ms
Wall time: 4.32 s
